# DB Initialisation Script

This notebook can be used for setting up local Postgres database called 'dsa3101' and for querying the database

To get started, please change the env variables in the next code chunk and simply 'Run All' to load the database.

Ensure that your .venv is activated in this Jupyter environment.

In [13]:
path_to_products_csv = 'products.csv'
path_to_ratings_csv = 'ratings.csv'
path_to_dailysales_csv = 'daily_sales.csv'
path_to_users_csv = 'users.csv'
path_to_user_behaviour_csv = 'user_behaviour.csv'
path_to_online_sales_csv = 'online_sales_edited.csv'
path_to_online_users_csv = 'online_sales_users.csv'
path_to_shipping_history_csv = 'shipping_history.csv'
path_to_shipping_status_csv = 'shipping_status.csv'

## Set Up

In [2]:
from sqlalchemy import create_engine, Table, Column, MetaData, String, Numeric, Float, Integer, ForeignKey, Date, ForeignKeyConstraint,DateTime
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os

In [9]:
# Load from .env file
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

load_dotenv(f'{parent_dir}/.env')

postgres_password = os.getenv('POSTGRES_PASSWORD')
postgres_port_no = os.getenv('POSTGRES_PORT')
host = os.getenv('POSTGRES_HOST')
database = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')

In [ ]:
# Step 1: Connect to the default database (postgres) to check/create the 'dsa3101' database
try:
    # Connect to the 'postgres' database (default)
    conn = psycopg2.connect(
        host=host,
        user=user,
        password=postgres_password,
        port=postgres_port_no,
        database='postgres'  # Connect to 'postgres' instead of 'dsa3101' initially
    )
    conn.autocommit = True  # Enable autocommit so that CREATE DATABASE works
    cursor = conn.cursor()

    # Check if the 'dsa3101' database exists
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{database}';")
    exists = cursor.fetchone()

    # If the database doesn't exist, create it
    if not exists:
        cursor.execute(f'CREATE DATABASE {database};')
        print(f"Database '{database}' has been created.")
    else:
        print(f"Database '{database}' already exists.")
    
    cursor.close()
    conn.close()

except Exception as e:
    print(f"Error while creating or checking the database: {e}")

# Step 2: Now connect to the newly created or existing 'dsa3101' database
try:
    engine = create_engine(f'postgresql://{user}:{postgres_password}@{host}:{postgres_port_no}/{database}')

    # Define MetaData for the DB
    metadata = MetaData()

    # Reflect existing tables from the 'dsa3101' database
    metadata.reflect(bind=engine)
    
    # Drop all tables in the 'dsa3101' database
    metadata.drop_all(bind=engine)
    print("All pre-existing tables have been dropped.")
    
except Exception as e:
    print(f"Error while reflecting or dropping tables: {e}")

finally:
    metadata = MetaData()


## Loading tables into database

In [11]:
# Define the 'products' table schema
products_table = Table('products', metadata,
    Column('product_id', String(50), primary_key=True),
    Column('product_name', String(500)),
    Column('about_product', String),  # Use String for TEXT in SQLAlchemy
    Column('category', String(255)),
    Column('actual_price', Numeric(10, 2)),
    Column('discounted_price', Numeric(10, 2)),
    Column('discount_percentage', Numeric(5, 2)),
    Column('origin_area', String(50)),
    Column('img_link', String(500))
)

# Define the 'ratings' table schema
ratings_table = Table('ratings', metadata,
    Column('product_id', String(50), ForeignKey('products.product_id', ondelete='CASCADE'), primary_key=True),
    Column('average_rating', Float),
    Column('review_title', String(1000)),
    Column('review_content', String),  # Use String for TEXT in SQLAlchemy
    Column('rating_count', Integer)
)

# Define the 'users' table schema
users_table = Table('users', metadata,
    Column('user_id', Integer, primary_key=True),
    Column('age', Integer),  
    Column('gender', String(50))
)

# Define the 'user_behaviour' table schema
user_behaviour_table = Table('user_behaviour', metadata,
    Column('timestamp', DateTime),
    Column('purchase_frequency', String(50)),
    Column('purchase_categories', String(255)),
    Column('personalized_recommendation_success', String(50)),
    Column('browsing_frequency', String(50)),
    Column('product_search_method', String(50)),
    Column('search_result_exploration', String(50)),
    Column('customer_reviews_importance', String(1)),  
    Column('add_to_cart_browsing', String(50)),
    Column('cart_completion_frequency', String(50)),
    Column('cart_abandonment_factors', String(255)),
    Column('saveforlater_frequency', String(50)),
    Column('review_left', String(50)),
    Column('review_reliability', String(50)),
    Column('review_helpfulness', String(50)),
    Column('recommendation_helpfulness', String(50)),
    Column('personalized_recommendation_frequency', String(1)),
    Column('rating_accuracy', String(1)),  
    Column('shopping_satisfaction', String(1)),  
    Column('service_appreciation', String(255)),
    Column('improvement_areas', String(255)),
    Column('user_id', Integer, primary_key=True),
    ForeignKeyConstraint(['user_id'], ['users.user_id'], ondelete='CASCADE')
)

# Define the 'online_sales' table schema
online_sales_table = Table('online_sales', metadata,
    Column('cust_id', Integer, primary_key=True),
    Column('transaction_id', Integer, primary_key=True),
    Column('date', Date),
    Column('product_id', String(50), primary_key=True),
    Column('coupon_status', String(50), primary_key=True),
    Column('coupon_code', String(50), primary_key=True),
    Column('discount_percentage', Numeric(5,2)),  
    Column('delivery_charges', Numeric(10, 2)),
    Column('quantity', Integer),
    ForeignKeyConstraint(['cust_id'], ['users.user_id'], ondelete='CASCADE'),
    ForeignKeyConstraint(['product_id'],['products.product_id'],ondelete='CASCADE')
)


# Define 'shipping_status' table schema
shipping_status_table = Table('shipping_status', metadata,
    Column('user_id', Integer),
    Column('transaction_id', Integer),
    Column('date', Date),
    Column('product_id', String(50)),
    Column('shipping_id', Integer, primary_key=True),
    Column('status', String(255)),
    Column('fulfilment', String(50)),
    Column('ship_service_level', String(50)),
    Column('estimated_delivery_date', Date),
    Column('fulfilled_by', String(255)),
    ForeignKeyConstraint(['user_id'], ['users.user_id'], ondelete='CASCADE'),
    ForeignKeyConstraint(['product_id'],['products.product_id'], ondelete='CASCADE'),
)

# Define the 'shipping_history' table schema
shipping_history_table = Table('shipping_history', metadata,
    Column('date', Date),
    Column('shipping_id', Integer, primary_key=True),
    Column('status', String(255),primary_key=True),
    Column('ship_service_level', String(50)),
    Column('update_date', Date),
    ForeignKeyConstraint(['shipping_id'],['shipping_status.shipping_id'], ondelete='CASCADE'),
)

# Create the table in the database if it doesn't exist
metadata.create_all(engine)


In [14]:
# Reading the csvs
products_df = pd.read_csv(path_to_products_csv)
ratings_df = pd.read_csv(path_to_ratings_csv)
users_df = pd.read_csv(path_to_users_csv)
user_behaviour_df = pd.read_csv(path_to_user_behaviour_csv)
online_sales_df = pd.read_csv(path_to_online_sales_csv)
online_users = pd.read_csv(path_to_online_users_csv)
shipping_history_df = pd.read_csv(path_to_shipping_history_csv)
shipping_status_df = pd.read_csv(path_to_shipping_status_csv)

# Convert 'date' column to correct format if necessary
online_sales_df['date'] = pd.to_datetime(online_sales_df['date'], format='%Y-%m-%d', errors='raise').dt.date
shipping_history_df['date'] = pd.to_datetime(shipping_history_df['date'], format='%Y-%m-%d', errors='raise').dt.date
shipping_status_df['date'] = pd.to_datetime(shipping_status_df['date'], format='%Y-%m-%d', errors='raise').dt.date
shipping_status_df['estimated_delivery_date'] = pd.to_datetime(shipping_status_df['estimated_delivery_date'], format='%Y-%m-%d', errors='raise').dt.date
shipping_history_df['update_date'] = pd.to_datetime(shipping_history_df['update_date'], format='%Y-%m-%d', errors='raise').dt.date

In [15]:
# Insert data into the PostgreSQL table
products_df.to_sql('products', engine, if_exists='append', index=False)
ratings_df.to_sql('ratings', engine, if_exists='append', index=False)
users_df.to_sql('users', engine, if_exists='append', index=False)
user_behaviour_df.to_sql('user_behaviour', engine, if_exists='append', index=False)
online_users.to_sql('users', engine, if_exists='append', index=False)
online_sales_df.to_sql('online_sales', engine, if_exists='append', index=False)
shipping_status_df.to_sql('shipping_status', engine, if_exists='append', index=False)
shipping_history_df.to_sql('shipping_history', engine, if_exists='append', index=False)

721